<a href="https://colab.research.google.com/github/DEP04929/ESICMDatathon2026/blob/main/ExtractingData_20260114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites for Amsterdam UMC DB


In [2]:
# sets *your* project id
PROJECT_ID = "esicmdatathon2026" #@param {type:"string"}


In [3]:
# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [4]:
import os
from google.colab import auth

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')


Authenticated


In [5]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 50000


In [6]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config


In [7]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns
sns.set_style('darkgrid')

# all possible ventilator patients
this will be used as criteria of person ids for all subsequent extractions

In [50]:
%%bigquery ventpatlist --project $PROJECT_ID
select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'

Query is running:   0%|          |

Downloading:   0%|          |

# ? Diagnosis: surgical or non surgical
All entries in condition occurrence are admission diagnosis. They are uncoded, so free text search is necessary.


In [56]:
%%bigquery surg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
), surg as (
select person_id, condition_start_datetime, condition_source_value
, case when lower(condition_source_value) like '%non%surg%' then 'non-surgical' else 'surgical' end as condition_type
from condition_occurrence
where condition_source_value like '%surg%' --matches surgical and non surgical
  and person_id in (select person_id from vlist)
)
select * from surg
PIVOT ( count(*) for condition_type in ('surgical', 'non-surgical'))
order by person_id, condition_start_datetime

Query is running:   0%|          |

Downloading:   0%|          |

In [57]:
surg.head(20)

,person_id,condition_start_datetime,condition_source_value,surgical,non-surgical
0,0,2010-01-01 00:03:00+00:00,Intracerebral subdural or subarachnoid haemorr...,0,1
1,0,2010-01-04 14:14:00+00:00,Neurological - Non surgical,0,1
2,5,2002-01-01 00:18:00+00:00,Heart valve surgery - Surgical,1,0
3,10,2010-01-01 02:00:00+00:00,Neurological - Non surgical,0,1
4,10,2010-04-09 02:20:00+00:00,Neurological - Non surgical,0,1
5,10,2010-04-25 12:34:00+00:00,Respiratory - Non surgical,0,1
6,11,2001-12-31 22:43:00+00:00,Coronary artery disease - Non surgical,0,1
7,11,2001-12-31 22:43:00+00:00,After cardiac arrest - surgical Ã« Non surgical,0,1
8,11,2001-12-31 22:43:00+00:00,Cardiogenic shock - Non surgical,0,1
9,21,2010-01-01 00:02:00+00:00,Bleeding - Non surgical,0,1


In [58]:
top_10_conditions = surg.groupby('condition_source_value')['person_id'].nunique().nlargest(10)
print(top_10_conditions)

condition_source_value
Heart valve surgery - Surgical                                       2397
Infection - Non surgical                                              922
Respiratory - Non surgical                                            862
After cardiac arrest - surgical Ã« Non surgical                       842
Peripheral vascular surgery - Surgical                                769
Neurological - Non surgical                                           672
Intracerebral subdural or subarachnoid haemorrhage - Non surgical     651
Sepsis - Surgical Ã« non surgical                                     613
Cardiovascular surgery, other                                         466
Gastrointestinal - Non surgical                                       455
Name: person_id, dtype: int64


# Heart rate

In [61]:
%%bigquery hf_ekg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
select person_id, measurement_datetime, value_as_number as hr_ekg
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in (
	21490872 --Heart rate.beat-to-beat by EKG
)
order by person_id, measurement_datetime

Query is running:   0%|          |

Downloading:   0%|          |

This shows the full range of all heart rate possibilities

In [62]:
hf_ekg.head(50)


,person_id,measurement_datetime,hr_ekg
0,3,2016-08-11 07:11:00+00:00,87.0
1,3,2016-08-11 12:57:53+00:00,88.0
2,3,2016-08-11 13:55:00+00:00,95.0
3,3,2016-08-11 14:16:00+00:00,91.0
4,3,2016-08-11 14:46:00+00:00,94.0
5,3,2016-08-11 15:16:00+00:00,90.0
6,3,2016-08-11 15:46:00+00:00,87.0
7,3,2016-08-11 16:16:00+00:00,86.0
8,3,2016-08-11 18:46:00+00:00,82.0
9,3,2016-08-12 10:02:00+00:00,96.0


# Arterial blood pressure

In [63]:
%%bigquery bp --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, bp as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 21490853 then 'ABPs'
     when measurement_concept_id = 21490851 then 'ABPd'
     when measurement_concept_id = 21490852 then 'ABPm'
--     when measurement_concept_id = 21492241 then 'NiBPm'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 21490851 -- ABP d
  , 21490852 -- ABP m
  , 21490853 -- ABPs
--  , 21492241 -- NiBPm
)
)
select * from bp
PIVOT ( max(value_as_number) for measure_type in ('ABPs', 'ABPd', 'ABPm')) --, 'NiBPm'
order by person_id, measurement_datetime

Query is running:   0%|          |

Downloading:   0%|          |

In [64]:
bp

,person_id,measurement_datetime,ABPs,ABPd,ABPm
0,0,2010-01-02 05:53:00+00:00,20265.0,NaN,NaN
1,3,2016-12-31 21:19:00+00:00,NaN,NaN,0.0
2,3,2016-12-31 21:20:00+00:00,NaN,NaN,0.0
3,3,2016-12-31 21:21:00+00:00,NaN,NaN,1.0
4,3,2016-12-31 21:22:00+00:00,NaN,NaN,1.0
...,...,...,...,...,...
8166435,65396,2017-01-06 15:00:00+00:00,138.0,58.0,85.0
8166436,65396,2017-01-06 16:00:00+00:00,123.0,55.0,78.0
8166437,65396,2017-01-06 17:00:00+00:00,139.0,69.0,92.0
8166438,65396,2017-01-06 18:00:00+00:00,138.0,65.0,91.0


# Breath rate or respiratory rate

Breath rate from ventilator and respiratory rate from monitor gets mixed up over the years of data recording.

In [65]:
%%bigquery rr --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, rr as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 3043148 then 'BRvent'
     when measurement_concept_id = 3026892 then 'BRspontv'
     when measurement_concept_id = 1175625 then 'BRspont'
     when measurement_concept_id = 3024171 then 'RR'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 3043148 --Breath rate mechanical
                                 , 3026892 -- Breath rate spontaneous --on ventilator
                                 , 1175625 -- Breath rate spontaneous
                                 , 3024171 -- Respiratory rate
                                 )
)
select * from rr
PIVOT ( max(value_as_number) for measure_type in ('BRvent', 'BRspontv','BRspont', 'RR'))
order by person_id, measurement_datetime

Query is running:   0%|          |

Downloading:   0%|          |

In [66]:
# rr[rr['BRvent']>90] # plausibility check
# rr[rr['person_id']==23444] # testing purpose, random person

# rr[rr['BRspont']>90] # plausibility check
rr[rr['person_id']==39444] # testing purpose, random person


,person_id,measurement_datetime,BRvent,BRspontv,BRspont,RR
7614476,39444,2016-12-31 22:43:00+00:00,NaN,NaN,NaN,34.0
7614477,39444,2017-01-01 00:27:00+00:00,NaN,NaN,35.8,37.0
7614478,39444,2017-01-01 01:27:00+00:00,NaN,NaN,0.0,30.0
7614479,39444,2017-01-01 01:57:00+00:00,NaN,NaN,1.9,NaN
7614480,39444,2017-01-01 02:27:00+00:00,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...
7616698,39444,2017-04-28 22:27:00+00:00,NaN,NaN,NaN,14.0
7616699,39444,2017-04-29 03:51:00+00:00,NaN,NaN,NaN,14.0
7616700,39444,2017-04-29 16:27:00+00:00,NaN,NaN,NaN,14.0
7616701,39444,2017-04-30 05:57:00+00:00,NaN,NaN,NaN,14.0


# BGA

In [67]:
%%bigquery bga --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
), br as ( select person_id, measurement_datetime
  , case when measurement_concept_id = 3010421 then 'pH'
     when measurement_concept_id = 3027315 then 'PaO2'
		 when measurement_concept_id = 3013290 then 'PaCO2'
		 when measurement_concept_id = 3006576 then 'HCO3'
		 when measurement_concept_id = 3012501 then 'BE'
		 when measurement_concept_id = 3047181 then 'Lactate'
     when measurement_concept_id = 3024928 then 'O2_venous'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
         3010421 -- pH of Blood
											,3027315 -- Oxygen [Partial pressure] in Blood
											,3013290 -- Carbon dioxide [Partial pressure] in Blood
											,3006576 -- Bicarbonate [Moles/volume] in Blood
											,3012501 -- Base excess in Blood by calculation
											,3047181 -- Lactate [Moles/volume] in Blood
                      ,3024928 -- Oxygen saturation in Venous blood
  )
  and value_as_number is not null
  and person_id in (select person_id from vlist)
  )
  select * from br
  PIVOT (max(value_as_number) for concept_name in ('Lactate', 'pH', 'PaO2', 'PaCO2', 'HCO3','BE','O2_venous'))
  order by person_id, measurement_datetime

Query is running:   0%|          |

Downloading:   0%|          |

In [68]:
bga

,person_id,measurement_datetime,Lactate,pH,PaO2,PaCO2,HCO3,BE,O2_venous
0,4,2017-01-01 00:32:00+00:00,NaN,NaN,141.0,NaN,NaN,NaN,NaN
1,4,2017-01-01 00:47:00+00:00,NaN,NaN,141.0,NaN,NaN,NaN,NaN
2,4,2017-01-01 03:02:00+00:00,NaN,NaN,82.5,NaN,NaN,NaN,NaN
3,4,2017-01-01 05:47:00+00:00,NaN,NaN,77.3,NaN,NaN,NaN,NaN
4,4,2017-01-01 06:02:00+00:00,NaN,NaN,77.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1096405,65396,2017-01-05 19:06:00+00:00,NaN,NaN,69.0,NaN,NaN,NaN,NaN
1096406,65396,2017-01-05 22:20:00+00:00,NaN,NaN,93.0,NaN,NaN,NaN,NaN
1096407,65396,2017-01-06 00:50:00+00:00,NaN,NaN,81.0,NaN,NaN,NaN,NaN
1096408,65396,2017-01-06 13:06:00+00:00,NaN,NaN,70.5,NaN,NaN,NaN,NaN


# PEEP

In [69]:
%%bigquery peep --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
 vent as
( select person_id, measurement_datetime
  , case when measurement_concept_id = 3012410 then 'TV'
       when measurement_concept_id = 2000000222 then 'MVspont'
       when measurement_concept_id = 3022875 then 'PEEP'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
        3012410 -- Tidal volume setting Ventilator
        , 3022875 -- Positive end expiratory pressure setting Ventilator
        , 2000000222 -- Minute volume spontaneous
  )
  and value_as_number is not null
  and person_id in (select person_id from vlist)
)
select * from vent
  PIVOT (max(value_as_number) for concept_name in ('MVspont','TV', 'PEEP'))
order by person_id, measurement_datetime

Query is running:   0%|          |

Downloading:   0%|          |

In [70]:
peep


,person_id,measurement_datetime,MVspont,TV,PEEP
0,3,2016-12-31 21:05:00+00:00,NaN,500.0,0.0
1,3,2016-12-31 21:06:00+00:00,NaN,500.0,0.0
2,3,2016-12-31 21:07:00+00:00,NaN,500.0,0.0
3,3,2016-12-31 21:08:00+00:00,NaN,500.0,0.0
4,3,2016-12-31 21:09:00+00:00,NaN,500.0,0.0
...,...,...,...,...,...
8121730,65396,2018-08-03 19:01:00+00:00,NaN,300.0,4.0
8121731,65396,2018-08-03 19:02:00+00:00,NaN,300.0,4.0
8121732,65396,2018-08-03 19:03:00+00:00,NaN,300.0,4.0
8121733,65396,2018-08-03 19:04:00+00:00,NaN,NaN,0.0


# FiO2

FiO2 exists for NIV too.

In [71]:
%%bigquery FiO2 --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
fio2 as (
select person_id, measurement_datetime
, case when measurement_concept_id in ( 3025408, 2000000204) then 'FiO2'
       when measurement_concept_id = 2000000203 then 'FiO2niv'
       end as concept_name
, value_as_number
from measurement m
where measurement_concept_id in (2000000203 -- FiO2 NIV
  , 3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
  --, 2000000204 -- fio2 setting
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from fio2
PIVOT (max(value_as_number) for concept_name in ('FiO2', 'FiO2niv'))
order by person_id, measurement_datetime

Query is running:   0%|          |

Downloading:   0%|          |

In [72]:
FiO2

,person_id,measurement_datetime,FiO2,FiO2niv
0,8,2017-01-01 04:14:00+00:00,59.0,NaN
1,8,2017-01-01 04:29:00+00:00,59.0,NaN
2,8,2017-01-01 04:44:00+00:00,47.0,NaN
3,8,2017-01-01 04:52:00+00:00,46.0,NaN
4,8,2017-01-01 05:44:00+00:00,44.0,NaN
...,...,...,...,...
710481,65392,2010-01-27 01:28:00+00:00,29.9,NaN
710482,65392,2010-01-27 02:28:00+00:00,30.0,NaN
710483,65392,2010-01-27 03:28:00+00:00,29.6,NaN
710484,65392,2010-01-27 04:28:00+00:00,29.5,NaN


# Labs: creatinine, bilirubin, hemoglobin and co.


In [84]:
%%bigquery lab --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
c as (
select person_id, measurement_datetime
, case when measurement_concept_id = 3020564 then 'Creatinine'
       when measurement_concept_id = 3005772 then 'Bilirubin'
       when measurement_concept_id = 40762351 then 'Hb'
       when measurement_concept_id = 3007461 then 'Platelets'
       when measurement_concept_id = 3010813 then 'wbc'
       end as concept_name
, value_as_number
from measurement m
where measurement_concept_id in (3020564 -- Creatinine [Moles/volume] in Serum or Plasma
  , 3005772 -- Bilirubin.conjugated [Moles/volume] in Serum or Plasma
  , 40762351 -- Hemoglobin [Moles/volume] in Blood'
  , 3007461 -- Platelets [#/volume] in Blood
  , 3010813 -- Leukocytes [#/volume] in Blood
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from c
PIVOT (max(value_as_number) for concept_name in ('Creatinine', 'Bilirubin', 'Hb', 'Platelets', 'wbc'))
order by person_id, measurement_datetime


Query is running:   0%|          |

Downloading:   0%|          |

In [85]:
lab

,person_id,measurement_datetime,Creatinine,Bilirubin,Hb,Platelets,wbc
0,0,2010-01-02 16:53:00+00:00,59.999999,NaN,NaN,NaN,NaN
1,0,2010-01-05 16:53:00+00:00,59.999999,NaN,NaN,NaN,NaN
2,3,2016-12-31 02:05:00+00:00,115.000000,NaN,8.7,311.0,7.5
3,4,2016-12-28 18:08:00+00:00,91.000000,NaN,8.5,315.0,9.8
4,4,2017-01-01 00:26:00+00:00,88.000000,NaN,NaN,242.0,15.1
...,...,...,...,...,...,...,...
711347,65396,2017-01-11 15:30:00+00:00,76.000000,NaN,7.9,215.0,11.5
711348,65396,2017-01-13 15:30:00+00:00,74.000000,NaN,7.7,233.0,10.1
711349,65396,2017-01-15 15:30:00+00:00,69.000000,NaN,7.2,231.0,10.8
711350,65396,2017-01-18 15:30:00+00:00,70.000000,NaN,7.8,269.0,7.1


# CAM score?

In [ ]:
%%bigquery test --project esicmdatathon2026
select *
from observation
where observation_concept_id in (3042082 --Confusion Assessment Method (CAM)
)
limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,observation_source_value,unit_source_value,value_source_value
0,13316,3042082,2006-08-07,2006-08-07 04:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
1,13316,3042082,2006-08-07,2006-08-07 04:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
2,13316,3042082,2006-08-07,2006-08-07 16:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
3,13316,3042082,2006-08-07,2006-08-07 16:33:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
4,57364,3042082,2006-12-10,2006-12-10 05:51:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
...,...,...,...,...,...,...,...,...,...
95,14704,3042082,2007-01-04,2007-01-04 04:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
96,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
97,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0
98,14704,3042082,2007-01-04,2007-01-04 16:27:00+00:00,0,0.0,CAM-ICU fluctuerend beloop,,0.0


In [ ]:
%%bigquery test --project esicmdatathon2026
select person_id, measurement_date, stddev_samp(value_as_number) as sd
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id in ( 21490872 --Heart rate.beat-to-beat by EKG
) and value_as_number > 0 and value_as_number < 300
group by person_id, measurement_date

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%%bigquery test --project $PROJECT_ID
select c.concept_name, m.measurement_source_value
, max(measurement_date) as last_date_used
, count(distinct person_id) as person_count
, count(*) as number_of_entries
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id = 3024171
group by concept_name, measurement_source_value
order by count(*) desc



Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
test

,concept_name,measurement_source_value,last_date_used,person_count,number_of_entries
0,Respiratory rate,AF totaal;R UMCA AN AF TOTAAL,2023-10-16,17485,6119517
1,Respiratory rate,AF (monitor);R UMCA ICU ADEMFREQUENTIE MONITOR,2022-06-16,21543,2377658
2,Respiratory rate,Respiratory Rate;R RESP RATE,2023-02-05,6349,2042237
3,Respiratory rate,Ademfreq.;ADEMHALING,2023-11-29,22390,1348806
4,Respiratory rate,Ademfreq.,2015-06-12,14263,1271302
5,Respiratory rate,AF (meet);R UMCA AN AF GEMETEN,2023-09-03,3286,1035294
6,Respiratory rate,AF prehosp.;R ED PRE-ARRIVAL RESP RATE,2023-07-01,1977,2296
7,Respiratory rate,Frequentie ingesteld;R UMCA NIV FREQUENTIE S,2021-03-01,83,2045
8,Respiratory rate,Frequentie gemeten;R UMCA NIV FREQUENTIE GEMETEN,2019-11-08,67,1688
9,Respiratory rate,Rate (Bipap Vision),2002-02-01,17,210


# Drugs? Vasopressors?

Quantity is there, but dose is not there, for drug exposure. I am unsure mg/ 24 hours unit meaning?  

In [74]:
%%bigquery vasopressor --project esicmdatathon2026
select person_id, drug_exposure_start_datetime, drug_exposure_end_datetime, drug_concept_id, quantity
from drug_exposure
where drug_concept_id in (19006871 -- argipressin
                      )
  limit 100

Query is running:   0%|          |

Downloading:   0%|          |

In [75]:
test

,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,drug_type_concept_id,quantity,drug_source_value,route_source_value,dose_unit_source_value
0,18116,19006871,2017-04-14,2017-04-14 09:56:00+00:00,NaT,NaT,38000180,0.015,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE/min
1,44731,19006871,2017-01-01,2017-01-01 14:00:00+00:00,NaT,NaT,38000180,0.020,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE/min
2,30325,19006871,2017-01-04,2017-01-04 05:35:00+00:00,NaT,NaT,38000180,0.010,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE/min
3,57649,19006871,2017-01-01,2017-01-01 01:41:00+00:00,NaT,NaT,38000180,0.010,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE/min
4,64616,19006871,2017-01-08,2017-01-08 12:40:00+00:00,NaT,NaT,38000180,0.010,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE/min
...,...,...,...,...,...,...,...,...,...,...,...
95,39729,19006871,2015-02-22,2015-02-22 14:47:00+00:00,2015-02-22,2015-02-22 15:47:00+00:00,38000180,0.780,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE
96,39729,19006871,2015-02-22,2015-02-22 16:47:00+00:00,2015-02-22,2015-02-22 17:29:00+00:00,38000180,0.462,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE
97,39729,19006871,2015-02-22,2015-02-22 18:47:00+00:00,2015-02-22,2015-02-22 19:21:00+00:00,38000180,0.102,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE
98,39729,19006871,2015-02-22,2015-02-22 04:47:00+00:00,2015-02-22,2015-02-22 05:47:00+00:00,38000180,1.800,"ARGIPRESSINE SPUITENPOMP 0,8 IE/ML IN NACL 0,9%",intraveneus,IE
